In [18]:
import cv2
import numpy as np
fileName='PC124surfTrim40.mp4'
cap = cv2.VideoCapture(fileName)

ret, old_frame = cap.read()

skipframe=50

for i in range (0,skipframe):
    ret, old_frame = cap.read()
    

cv2.imwrite("old_frame.jpg", old_frame)
cap.release()   

%run click_and_crop.py --image old_frame.jpg

In [23]:
import cv2
import numpy as np

#Importar video

fileName='PC124surfTrim40.mp4'
cap = cv2.VideoCapture(fileName)

ret, old_frame = cap.read()

#########################################################
skipframe=50

for i in range (0,skipframe):
    ret, old_frame = cap.read()
###########################################################



height , width , layers =  old_frame.shape

salida = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'XVID'), 30,(width,height))
############################################################################################################################
#Click to take template

template = cv2.imread('template.jpg')
#############################################################################################################################


#primera deteccion de rectangulo
#img = cv2.cvtColor(old_frame,cv2.COLOR_BGR2GRAY)
img = old_frame
w, h ,l = template.shape[::-1]

# Apply template Matching
res = cv2.matchTemplate(img,template,cv2.TM_CCOEFF)

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(img,top_left, bottom_right, 255, 2)

top_left_original = top_left
bottom_right_original = bottom_right

while(1):
    #img = cv2.cvtColor(old_frame,cv2.COLOR_BGR2GRAY)
    img = old_frame
    
    img2 = img.copy()
    w, h ,l= template.shape[::-1]
    # All the 6 methods for comparison in a list
    #methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
    #            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

    #img = img2.copy()
    # Apply template Matching
    res = cv2.matchTemplate(img,template,cv2.TM_CCOEFF_NORMED)
    
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img,top_left, bottom_right, 255, 2)

    
    #transformacion para estabilizacion
    tx1 = top_left_original[0]-top_left[0]
    ty1 = top_left_original[1]-top_left[1]
    tx2 = bottom_right_original[0]-bottom_right[0]
    ty2 = bottom_right_original[1]-bottom_right[1]
    
    tx = (tx1+tx2)//2+500
    ty = (ty1+ty2)//2-300

    num_rows, num_cols = img.shape[:2]

    translation_matrix = np.float32([ [1,0,tx], [0,1,ty] ])
    img_translation = cv2.warpAffine(img2, translation_matrix, (num_cols, num_rows))
    
    img_translation=cv2.resize(img_translation,(width,height))
    
    salida.write(img_translation)
    
    cv2.imshow('Matching Result',res)
    cv2.imshow('Detected Point',img)
    cv2.imshow('Estabilizada',img_translation)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
    
    ret, old_frame = cap.read()
    if ret==0:
        break
    
cv2.destroyAllWindows()
salida.release()
cap.release()    